In [177]:
from pyspark.sql import SparkSession
import datetime as dt
import pyspark.sql.functions
spark = SparkSession.builder.appName('etl').getOrCreate()
df = spark.read.parquet('data/catalog.parquet')

## Measurement & Observation mapping

In [178]:
filtered = df.filter(df['resourceType'] == 'Observation')

In [179]:
Measurement = filtered.filter(filtered.valueCodeableConcept.isNull())
Observation = filtered.filter(filtered.valueCodeableConcept.isNotNull())

Measurement = filtered.select(['id',
                            'subject',
                            'code',
                            'performer',
                            'encounter',
                            'meta',
                            'valueQuantity',
                            'effectiveDateTime',
                            'Extension.valueCodeableConcept'])

#Measurement.toPandas().to_csv("dfdfa.csv", header=True)

 #### TODO
[x] Split into Observation/Measurement :
   * Measurements have valueQuantity     
   * Observations (should) have valueString
     
     
[] Rename Columns

In [180]:
split_col = pyspark.sql.functions.split(Measurement["effectiveDateTime"], 'T')

Measurement = Measurement.withColumnRenamed("id", "measurement_id")\
                                .withColumn("measurement_date", split_col.getItem(0))\
                                .withColumnRenamed("subject", "person_id")\
                                .withColumnRenamed("code", "measurement_concept_id")\
                                .withColumnRenamed("effectiveDateTime", "measurement_datetime")\
                                .drop("valueCodeableConcept")\
                                .withColumnRenamed("category", "measurement_type_concept_id")\
                                .withColumn("value_as_number", Measurement.valueQuantity.value)\
                                .withColumnRenamed("encounter", "visit_occurrence_id")\
                                .withColumn("unit_source_value", Measurement.valueQuantity.unit)\
                                .withColumnRenamed("performer", "provider_id")\
                                .drop("valueQuantity")


# Measurement.toPandas().to_csv("measurememts.csv", header=True)

Measurement.show(1)

+--------------------+--------------------+----------------------+-----------+--------------------+--------------------+--------------------+----------------+---------------+-----------------+
|      measurement_id|           person_id|measurement_concept_id|provider_id| visit_occurrence_id|                meta|measurement_datetime|measurement_date|value_as_number|unit_source_value|
+--------------------+--------------------+----------------------+-----------+--------------------+--------------------+--------------------+----------------+---------------+-----------------+
|3a85e0c5-c469-42c...|{urn:uuid:46c9970...|  {[{http://loinc.o...|       null|{urn:uuid:ef55a39...|{2020-12-15T20:38...|2011-12-01T03:37:...|      2011-12-01|{0.13747, null}|             kU/L|
+--------------------+--------------------+----------------------+-----------+--------------------+--------------------+--------------------+----------------+---------------+-----------------+
only showing top 1 row

